In [1]:
# NO TERMINAL
# pip install earthengine-api
# pip install geemap
# pip install pywebhdfs
# pip install geopandas

In [2]:
# DOWNLOAD FROM GEE

In [10]:
import ee
import geemap
from pywebhdfs.webhdfs import PyWebHdfsClient
from datetime import date, timedelta
import os
import geopandas as gpd

In [4]:
# Inicia o fluxo de autenticação
ee.Authenticate()

# Inicializa a biblioteca
ee.Initialize()

Enter verification code:  4/1AX4XfWgChJ0zb8Moe0TBT-joyNm6DNKHo6L18a0TkCyKTSFgnOFsGSwn92A



Successfully saved authorization token.


In [12]:
limites = gpd.read_file('inputSentinelExample/limites.shp')
filter_field = 'id_pk'
feature_names = list(set([v[filter_field] for k, v in limites.iterrows()]))

In [13]:
# VARIAVEIS GLOBAIS (MUDAR AQUI!)
end_date = date.today()
percent_cloud = 100
# box = [[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin], [xmin, ymin]]
# https://boundingbox.klokantech.com/
# box = [[[-54.6306579887,-25.5892766534],[-54.5393341362,-25.5892766534],[-54.5393341362,-25.4046299874],[-54.6306579887,-25.4046299874],[-54.6306579887,-25.5892766534]]]
# boundary = ee.Geometry.Polygon(box, None, False)
collection_name = "COPERNICUS/S2_SR"
scale = 10
crs = 'EPSG:4326'
# CRIE UMA PASTA VAZIA NO DIRETORIO RASTER E COLOQUE O CAMINHO DELA AQUI EM BAIXO
out_dir = "raster/sentinel2_tmp"
hdfs_dir = "sentinel2_tmp"

In [16]:
for feature_name in feature_names:
    out_dir_feature = out_dir+"/"+str(feature_name)
    filter_polygon = limites.loc[(limites[filter_field] == feature_name)]
    xmin, ymin, xmax, ymax = filter_polygon.total_bounds
    box = [[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin], [xmin, ymin]]
    boundary = ee.Geometry.Polygon(box, None, False)
    for i in range(2, 15, 2):
        start_date = str((end_date - timedelta(days=i)))
        collection = ee.ImageCollection(collection_name) \
            .select(['B2', 'B3', 'B4', 'B8']) \
            .filterBounds(boundary) \
            .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', percent_cloud) \
            .filterDate(start_date, str(end_date))
        geemap.ee_export_image_collection(collection, scale=scale, crs=crs, region=boundary, out_dir=out_dir_feature)
    #     NOT SURE WHAT F.. THIS BELOW DOES 
        for root, directory, files in os.walk(out_dir):
                    if len(files) == 0:
                        geemap.ee_export_image_collection(collection, scale=scale, crs=crs, region=boundary, out_dir=out_dir_feature)
                    else:
                        print('Existem imagens na pasta')


Total number of images: 0

Total number of images: 0

Total number of images: 0

Total number of images: 2

Exporting 1/2: 20211226T134211_20211226T134212_T21JYN.tif
Generating URL ...
Please wait ...
Data downloaded to /opt/workspace/raster/sentinel2_tmp1/20211226T134211_20211226T134212_T21JYN.tif


Exporting 2/2: 20211226T134211_20211226T134212_T21KYP.tif
Generating URL ...
Please wait ...
Data downloaded to /opt/workspace/raster/sentinel2_tmp1/20211226T134211_20211226T134212_T21KYP.tif


Total number of images: 2

Exporting 1/2: 20211226T134211_20211226T134212_T21JYN.tif
Generating URL ...
Please wait ...
Data downloaded to /opt/workspace/raster/sentinel2_tmp1/20211226T134211_20211226T134212_T21JYN.tif


Exporting 2/2: 20211226T134211_20211226T134212_T21KYP.tif
Generating URL ...
Please wait ...
Data downloaded to /opt/workspace/raster/sentinel2_tmp1/20211226T134211_20211226T134212_T21KYP.tif


Total number of images: 2

Exporting 1/2: 20211226T134211_20211226T134212_T21JYN.tif
Gene

KeyboardInterrupt: 

In [ ]:
hdfs = PyWebHdfsClient(host='179.106.229.159',port='50070', user_name='root')
hdfs.delete_file_dir(hdfs_dir, recursive=True)
hdfs.make_dir(hdfs_dir)

In [ ]:
# Adicionar imagens no HDFS

# No terminal do JupyterLab
# sudo docker exec -itu 0 hadoop passwd (primeira vez) - hoje é admin123
# ssh hadoop
# cd ..
# cd hadoop/bin
# ./hadoop fs -copyFromLocal /opt/workspace/raster/sentinel2_tmp/* /sentinel2_tmp